# このノートブックについて

autoencoderを用いて特徴抽出したものを、Kmeansを使ってクラスタリングする。  
その結果を病気の診断に用いれるかを検討した。  

# インポート+データ準備+モデルロード

In [64]:
from Functions import process_img, get_labels
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd

from keras import backend as K
from keras.layers import Dense, Flatten
from keras.models import Sequential, load_model
from keras.applications import VGG16
from keras.optimizers import Adam

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.cluster import k_means

In [6]:
path1 = "./dataset/data_reshape/image_onehand_reshape/N"
path2 = "./dataset/data_reshape/image_onehand_reshape/H"
path3 = "./dataset/data_reshape/image_onehand_reshape/R"

N_imgtri_list = glob.glob(path1 + "/*" + ".png")
print(len(N_imgtri_list))
H_imgtri_list = glob.glob(path2 + "/*" + ".png")
print(len(H_imgtri_list))
R_imgtri_list = glob.glob(path3 + "/*" + ".png")
print(len(R_imgtri_list))

N_arr = process_img(N_imgtri_list)
H_arr = process_img(H_imgtri_list)
R_arr = process_img(R_imgtri_list)

dataset = np.concatenate([N_arr, H_arr, R_arr], axis=0)
print(dataset.shape)

t = get_labels(27, 40, 49)
print(t.shape)

x_train, x_test, t_train, t_test = train_test_split(dataset, t, test_size=0.2)

27
40
49
(116, 224, 224, 3)
(116, 3)


In [14]:
unet = load_model("./model/segmentation/unet_224_pred2.h5")

# 推定

In [32]:
pred = unet.predict(dataset)

In [34]:
pred.shape

(116, 224, 224, 1)

In [63]:
# kmeanでクラスタリングするため1dimにする
pred_1dim = pred.reshape(116, -1)

In [36]:
pred_1dim.shape

(116, 50176)

# クラスタリング

In [37]:
# 2クラスター分類
kmean = k_means(pred_1dim, n_clusters=2)

In [43]:
# データの前半は正常、後半を病気としてそれぞれのクラスターを取り出す
N_cluster = kmean[1][:27]
Sick_cluster = kmean[1][28:]

In [56]:
# 正常データで２クラスター分類した結果から、正解しているものの数
N_cluster_ok = np.sum(N_cluster == 0)
N_cluster_ok

21

In [57]:
# 病気データで２クラスター分類した結果から、正解しているものの数
Sick_cluster_ok = np.sum(Sick_cluster == 1)
Sick_cluster_ok

51

In [65]:
result_dict = {"正常[予測]": [21, 38], "病気[予測]": [6, 51]}

In [67]:
conf = pd.DataFrame(result_dict, index = ["正常[実際]", "病気[実際]"])
conf

,正常[予測],病気[予測]
正常[実際],21,6
病気[実際],38,51


In [70]:
acc = (21 + 51) / pred_1dim.shape[0]
acc

0.6206896551724138

# 考察

正解率0.62であった。  
精度が低いが、それよりも良くないのが予測正常で実際は病気である割合が問題だ。  
使うことはできないだろう。